In [ ]:
import os
import re
import json
import pandas as pd

from pathlib import Path


In [ ]:
# -----------------------------------------------------
# 경로 설정
# -----------------------------------------------------

def find_project_root() -> Path:
    p = Path.cwd()

    for parent in [p] + list(p.parents):
        if (parent / "data").exists() and (parent / "notebooks").exists():
            return parent

    return p

PROJECT_ROOT = find_project_root()

TREND_DIR = PROJECT_ROOT / "data" / "raw" / "kaggle" / "youtube_trending_video_dataset"
if not TREND_DIR.exists():
    raise FileNotFoundError(f"트렌딩 데이터 폴더가 없습니다: {TREND_DIR}")

JSON_BASE_DIR = PROJECT_ROOT / "configs" / "categories"
JSON_BASE_DIR.mkdir(parents=True, exist_ok=True)

OUT_DIR = PROJECT_ROOT / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT :", PROJECT_ROOT)
print("TREND_DIR    :", TREND_DIR)
print("JSON_BASE_DIR:", JSON_BASE_DIR)
print("OUT_DIR      :", OUT_DIR)


In [ ]:
# -----------------------------------------------------
# 다국가 CSV 자동 탐색
# -----------------------------------------------------

paths = {}

for csv_path in TREND_DIR.glob("*_youtube_trending_data.csv"):
    cc = csv_path.name.split("_", 1)[0]

    if re.fullmatch(r"[A-Z]{2}", cc):
        paths[cc] = csv_path

print("Loaded countries:", sorted(paths.keys()))
print("Total countries:", len(paths))

if not paths:
    raise FileNotFoundError(f"*_youtube_trending_data.csv 파일을 찾지 못했습니다: {TREND_DIR}")

# -----------------------------------------------------
# 국가별 category json 경로 탐색
# -----------------------------------------------------

def find_category_json(country_code: str) -> Path | None:
    p1 = JSON_BASE_DIR / country_code / f"{country_code}_category_id.json"
    if p1.exists():
        return p1

    p2 = JSON_BASE_DIR / f"{country_code}_category_id.json"
    if p2.exists():
        return p2

    return None

category_json_paths = {cc: find_category_json(cc) for cc in paths.keys()}
missing_json = [cc for cc, jp in category_json_paths.items() if jp is None]

print("Category JSON missing:", missing_json[:10], f"(+{max(0, len(missing_json)-10)} more)" if len(missing_json) > 10 else "")

# -----------------------------------------------------
# 파일 누락 체크
# -----------------------------------------------------

missing = []

for country, path in paths.items():

    if not os.path.exists(path):
        missing.append((country, path))

if missing:
    print("누락된 CSV 파일:")

    for country, path in missing:
        print(f" - {country}: {path}")

    raise FileNotFoundError("일부 국가 CSV 파일이 누락되어 실행을 중단합니다.")

else:
    print("모든 국가 CSV 파일이 정상적으로 존재합니다.")


모든 국가 CSV 파일이 정상적으로 존재합니다.


In [ ]:
# -----------------------------------------------------
# 필요한 컬럼 선택
# -----------------------------------------------------

use_cols = [
    "channelId", "video_id", "title",
    "publishedAt", "trending_date",
    "categoryId", "tags",
    "view_count", "likes", "comment_count"
]

# -----------------------------------------------------
# CSV 읽기 + 검증 + 전처리
# -----------------------------------------------------

yt_trending_list = []  # 리스트 초기화

for country, path in paths.items():

    # CSV 읽기
    try:
        df = pd.read_csv(path, low_memory=False, encoding="latin1")

    except Exception as e:
        print(f"CSV 형식 오류 발생 ({country}): {path}")

        raise ValueError(f"{country} CSV 파일을 읽는 중 오류 발생 → {str(e)}")

    # 필수 컬럼 누락 검사
    missing_cols = [col for col in use_cols if col not in df.columns]

    if missing_cols:
        raise ValueError(f"{country} CSV에서 필수 컬럼 누락: {missing_cols}")
        raise ValueError(f"{country} CSV에서 필수 컬럼이 누락되어 실행을 중단합니다.")

    print(f"{country}: 필수 컬럼 정상 확인")

    # 필요한 컬럼만 추출
    df = df[use_cols]

    # 국가 컬럼 추가
    df["country"] = country

    # 리스트에 추가
    yt_trending_list.append(df)

print("--------------------")
print("모든 CSV 컬럼 검증 완료")


BR: 필수 컬럼 정상 확인
CA: 필수 컬럼 정상 확인
DE: 필수 컬럼 정상 확인
FR: 필수 컬럼 정상 확인
GB: 필수 컬럼 정상 확인
IN: 필수 컬럼 정상 확인
JP: 필수 컬럼 정상 확인
KR: 필수 컬럼 정상 확인
MX: 필수 컬럼 정상 확인
RU: 필수 컬럼 정상 확인
US: 필수 컬럼 정상 확인
--------------------
모든 CSV 컬럼 검증 완료


In [ ]:
# -----------------------------------------------------
# (중복 제거) yt_trending_list 병합
# -----------------------------------------------------

if "yt_trending_list" not in globals() or len(yt_trending_list) == 0:
    raise ValueError("yt_trending_list가 비어있습니다. 셀 #4가 정상 실행되었는지 확인하세요.")

yt_trending_df = pd.concat(yt_trending_list, ignore_index=True)

print("✅ merged shape:", yt_trending_df.shape)
display(yt_trending_df.head())


병합 완료: (2905678, 11)


In [ ]:
# ----------------------------------------------
# 날짜 변환
# ----------------------------------------------

yt_trending_df["publishedAt"] = pd.to_datetime(yt_trending_df["publishedAt"], errors="coerce")
yt_trending_df["trending_date"] = pd.to_datetime(yt_trending_df["trending_date"], errors="coerce")

# 날짜 결측치 제거
yt_trending_df = yt_trending_df.dropna(subset=["publishedAt", "trending_date"])


In [ ]:
# ----------------------------------------------
# tags 파싱
# ----------------------------------------------

yt_trending_df["tags"] = yt_trending_df["tags"].fillna("")

def parse_tags(x):

    if x in ["[none]", "None", "none", ""]:
        return []
    
    return x.split("|")

yt_trending_df["tags_list"] = yt_trending_df["tags"].apply(parse_tags)
yt_trending_df["tags_count"] = yt_trending_df["tags_list"].apply(len)


In [ ]:
# ----------------------------------------------
# trending_days 계산
# ----------------------------------------------

yt_trending_df["trending_days"] = yt_trending_df.groupby("video_id")["video_id"].transform("count")


In [ ]:
# ----------------------------------------------
# 날짜 기반 파생 컬럼
# ----------------------------------------------

yt_trending_df["publish_month"] = yt_trending_df["publishedAt"].dt.month
yt_trending_df["publish_dayofweek"] = yt_trending_df["publishedAt"].dt.dayofweek
yt_trending_df["days_since_publish"] = (yt_trending_df["trending_date"] - yt_trending_df["publishedAt"]).dt.days


In [ ]:
# ----------------------------------------------
# 비율 기반 파생 컬럼
# ----------------------------------------------

yt_trending_df["like_ratio"] = yt_trending_df["likes"] / yt_trending_df["view_count"].replace(0, 1)
yt_trending_df["comment_ratio"] = yt_trending_df["comment_count"] / yt_trending_df["view_count"].replace(0, 1)
yt_trending_df["engagement_score"] = (yt_trending_df["likes"] + yt_trending_df["comment_count"]) / yt_trending_df["view_count"].replace(0, 1)


In [ ]:
# ----------------------------------------------
# categoryId → category_name 자동 매핑
# ----------------------------------------------

def load_category_map(json_path: str) -> dict:
    if not json_path or not os.path.exists(json_path):
        return {}

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    mapping = {}
    for item in data.get("items", []):
        try:
            cid = int(item["id"])
            name = item["snippet"]["title"]
            mapping[cid] = name
        except Exception:
            continue

    return mapping

# 전체 국가 category JSON 자동 읽기
category_map = {}

if os.path.exists(JSON_BASE_DIR):
    for country in paths.keys():
        p1 = os.path.join(JSON_BASE_DIR, country, f"{country}_category_id.json")
        p2 = os.path.join(JSON_BASE_DIR, f"{country}_category_id.json")
        json_path = p1 if os.path.exists(p1) else (p2 if os.path.exists(p2) else None)

        if json_path:
            category_map.update(load_category_map(json_path))

# categoryId 타입 보정
yt_trending_df["categoryId"] = (
    pd.to_numeric(yt_trending_df["categoryId"], errors="coerce")
    .fillna(-1)
    .astype(int)
)

# category_name 생성
yt_trending_df["category_name"] = yt_trending_df["categoryId"].map(category_map).fillna("Unknown")

print("카테고리 매핑 로딩 완료 (총 개수:", len(category_map), ")")
print("category_name 예시:", yt_trending_df["category_name"].dropna().unique()[:10])


카테고리 매핑 로딩 완료 (총 개수: 0 )


In [ ]:
# -----------------------------------------------------
# 파생 컬럼 검증
# -----------------------------------------------------

derived_cols = [
    "trending_days", "tags_list", "tags_count", "category_name",
    "publish_month", "publish_dayofweek", "days_since_publish",
    "like_ratio", "comment_ratio", "engagement_score"
]

missing_derived = [col for col in derived_cols if col not in yt_trending_df.columns]

if missing_derived:
    raise ValueError(f"파생 컬럼이 생성되지 않음: {missing_derived}")

print("파생 컬럼 검증 완료")


파생 컬럼 검증 완료


In [ ]:
# -----------------------------------------------------
# 저장 경로 자동 생성
# -----------------------------------------------------

def get_next_version_file(out_dir: Path, base_name: str, ext: str = "csv") -> Path:
    out_dir.mkdir(parents=True, exist_ok=True)

    pattern = re.compile(rf"^{re.escape(base_name)}_v(\d+)\.{re.escape(ext)}$")
    versions = []

    for f in out_dir.iterdir():
        if f.is_file():
            m = pattern.match(f.name)
            if m:
                versions.append(int(m.group(1)))

    next_version = max(versions) + 1 if versions else 1
    
    return out_dir / f"{base_name}_v{next_version}.{ext}"


In [ ]:
# -----------------------------------------------------
# 저장
# -----------------------------------------------------

base_name = "trending_videos_clean"
save_path = get_next_version_file(OUT_DIR, base_name, ext="csv")

yt_trending_df.to_csv(save_path, index=False, encoding="utf-8-sig")

print("✅ saved:", save_path)
print("✅ rows :", len(yt_trending_df))
print("✅ countries:", sorted(paths.keys()))
print("✅ category_name examples:", yt_trending_df["category_name"].dropna().unique()[:10])


저장 완료: C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\YT_ChannelGrowth_Engagement\data\processed\youtube_trending_video_clean_v2.csv
최종 데이터 형태: (2905678, 21)
